# Statistical Language model 
$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\def\c#1{\mathcal{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$


One important application of Markov models is to create language models (LM), which are models
which can generate (or score) a sequence of words.

## How to score a sentence?
For example, the sentence "I like to eat apple." is treated as list ['I', 'like', 'to', 'eat', 'apple']. Then the probability to score a sentence becomes to find the probability of the list
$$ P(\text{I like to eat apple}) = P(\text{'I', 'like', 'to', 'eat', 'apple'})$$

We need to collect lots of text data from news, article, or even wikipedia. The number of vocabulary in daily use is roughtly 5000 words.

- We need $5000^n$ text data to sufficiently calculate the probability for the sentence of length n. $n=2,3,4,...$.

- If you randomly generate a sentence, the probability that sentence makes sense is extremely small or even 0.

- So to judge a sentence makes sense, we only need a relative probability. It is sufficient as far as our model can say the following (or similar) statement.
$$P(\text{'I', 'like', 'to', 'eat', 'apple'}) >P(\text{'apple', 'likes', 'to', 'eat', 'me'}) $$

- Instead of studying the whole sequence, we can focus on local connection. For example, it is more likely to find 'eat apple'
than 'apple eats', i.e.,
$$ P(\text{apple}|\text{eat}) > P(\text{eat}|\text{apple})$$


- When we use a finite-state Markov model with
a memory of length $m=n-1$. it is called an $n$-gram model.

   - If $m = 1$, we get a unigram model (no dependence on previous words);

   - If $m = 2$, we get a bigram model (depends
on previous word);

   - If $m = 3$, we get a trigram model (depends on previous two words);

- Most n-grams will never be observed, so you’ll have lots of
zero probability n-grams. This is an example of data sparsity. Your model depends increasingly on the training data; you
need (lots) more data to learn to generalize well.

- These days, most LMs are built using recurrent neural nets, which have
unbounded memory. However, simple n-gram models can still do quite well when trained with enough data.


### Example
The sentence is 'I saw the red house'.  $\langle s\rangle$ is the starting of the sentence and $\langle /s\rangle$ is rhe ending of the sentence.

- In bigram model,
\begin{align}
& P(\text{I, saw, the, red, house}) \\
\approx {} & P(\text{I}\mid\langle s\rangle) P(\text{saw}\mid \text{I}) P(\text{the}\mid\text{saw}) P(\text{red}\mid\text{the}) P(\text{house}\mid\text{red}) P(\langle /s\rangle\mid \text{house})
\end{align}


- In trigram model,
\begin{align}
& P(\text{I, saw, the, red, house}) \\
\approx {} & P(\text{I}\mid \langle s\rangle,\langle s\rangle) P(\text{saw}\mid\langle s\rangle,I) P(\text{the}\mid\text{I, saw}) P(\text{red}\mid\text{saw, the}) P(\text{house}\mid\text{the, red}) P(\langle /s\rangle\mid\text{red, house})
\end{align}



## Evaluation of Language Models

Given a test dataset $X$ (of $N$ words), we arrive at the standard
intrinsic evaluation in three steps:
- Probability of the test data: $p(X, \theta)$.

- That value will be tiny, because total of words
is infinitely large, and $P$ will decrease exponentially in the length of $X$. So we take a negated log and divide by the number of vocabulary:
\begin{align}
\text{CrossEntropy}(p(\cdot; \theta), X)=\frac{-\log_2 p(X; \theta)}{N}
\end{align}
You can interpret cross-entropy in “bits per word.” Lower is
better.

- Perplexity is $2^{\text{CrossEntropy}(p(\cdot; \theta), X)}$. You can interpret perplexity as “effective size of the
vocabulary.
 Special cases:
  - If the model were to put all of its probability on $X$, perplexity would be 1 (minimal possible value).

  - If the model assigns zero probability to $X$, perplexity is $+\infty$.
So it’s important to make sure that $p$ assigns strictly positive
probability to every sequence of words.


- Warning: you can only compare perplexity of models that use
exactly the same vocabulary. We can also measure perplexity on the training data. Do you expect training perplexity to be lower (i.e., better) than test perplexity, or higher (i.e., worse)? Why?



In [3]:
%pylab inline
import re
import string
import requests
import collections
from collections import Counter


Populating the interactive namespace from numpy and matplotlib


In [4]:
class SimpleMarkovModel(object):
    def __init__(self, status_num=None):
        # initial probability vector
        self.pi = np.zeros(shape=(status_num))
        # transition probability matrix
        self.P = np.zeros(shape=(status_num, status_num))

    def fit(self, x):
        """
        Based on training data, calculate initial probability vector and transition probability matrix
        param x: x can be a single list or list of list. [s1, s2, ..., sn] or [[s11,s12,...,s1m],[s21,s22,...,s2n],...]
        The difference is in calculating initial probability vector. In single list, we can use all states to inference the initial
        prob vector. In list of list, we will use the initial states of sub-list to inference.
        return:
        """
        if type(x[0]) == list:
            for clist in x:
                self.pi[clist[0]] += 1
                for cindex in range(0, len(clist) - 1):
                    self.P[clist[cindex ], clist[cindex + 1]] += 1
        else:
            for index in range(0, len(x) - 1):
                self.pi[x[index]] += 1
                self.P[x[index ], x[index + 1]] += 1
        # normalization
        self.pi = self.pi / np.sum(self.pi)
        normalization = np.sum(self.P, axis=1) # <--
        normalization[normalization == 0] = 1  # <--
        self.P = self.P / normalization[:, np.newaxis]


    def predict_log_joint_prob(self, status_list, set_init_prob=None):
        """
        calculate the log of the joint probability
        param: status_list:
        :return:
        """
        prob = self.pi if set_init_prob is None else set_init_prob

        log_prob = np.log(prob[status_list[0]])
        for index in range(0, len(status_list) - 1):
            log_prob += np.log(self.P[status_list[index], status_list[index + 1]])
        return log_prob


    def predict_prob_distribution(self, time_steps=None, set_init_prob=None, set_prob_trans_matrix=None):
        """
        calculate the prob distribution after time_steps.
        Allow set_init_prob and set_prob_trans_matrix to set initial prob and prob_trans_matrix
        :param time_steps:
        :param set_init_prob:
        :param set_prob_trans_matrix:
        :return:
        """
        prob = self.pi if set_init_prob is None else set_init_prob
        trans_matrix = self.P if set_prob_trans_matrix is None else set_prob_trans_matrix
        for _ in range(0, time_steps):
            prob = np.matmul(prob, trans_matrix)
        return prob

    def predict_next_step_prob_distribution(self, current_status=None):
        """
        predict next step probility distribution
        :param current_status:
        :return:
        """
        return self.P[[current_status], :]

    def predict_next_step_status(self, current_status=None):
        """
        predict the most probable state in the next step
        :param current_status:
        :return:
        """
        return np.argmax(self.predict_next_step_prob_distribution(current_status))


    def generate_status(self, step_times=10, stop_status=None, set_start_status=None, search_type="greedy", beam_num=5):
        """
        greedy search and beam search
        :param step_times: maximum step_times
        :param stop_status: list of stoping state
        :param set_start_status: set the initial state manually
        :param search_type: greedy and beam
        :param beam_num: only when search_type="beam" to keep top k
        :return:
        """
        if stop_status is None:
            stop_status = []

        start_status = np.random.choice(len(self.pi.reshape(-1)),
                                        p=self.pi.reshape(-1)) if set_start_status is None else set_start_status
        if search_type == "greedy":

            rst = [start_status]
            for _ in range(0, step_times):
                next_status = self.predict_next_step_status(current_status=start_status)
                rst.append(next_status)
                if next_status in stop_status:
                    break
                start_status = next_status

        else:
            # beam search
            rst = [start_status]
            top_k_rst = [[start_status]]
            top_k_prob = [0.0]
            for _ in range(0, step_times):
                new_top_k_rst = []
                new_top_k_prob = []
                for k_index, k_rst in enumerate(top_k_rst):
                    k_rst_last_status = k_rst[-1]
                    # get top k largest idx
                    top_k_idx = self.P[k_rst_last_status, : ].argsort()[::-1][0:beam_num]
                    for top_k_status in top_k_idx:
                        new_top_k_rst.append(k_rst + [top_k_status])
                        new_top_k_prob.append(top_k_prob[k_index] + np.log(1e-12+self.P[k_rst_last_status, top_k_status]))
                # sort all beam_num*beam_num results and get the top beam_num
                top_rst_idx = np.asarray(new_top_k_prob).argsort()[::-1][0:beam_num]
                rst = new_top_k_rst[top_rst_idx[0]]
                # update
                top_k_rst = []  break
                    else:
                        top_k_rst.append(new_top_k_rst[top_idx])
                        top_k_prob.append(new_top_k_prob[top_idx])

        return rst




In [5]:
url = "https://raw.githubusercontent.com/probml/probml-data/main/data/timemachine.txt"
response = requests.get(url)
data = response.text
lines = [s + "\n" for s in response.text.split("\n")]
raw_dataset = [re.sub("[^A-Za-z]+", " ", st).lower().split() for st in lines]

# Print first few lines
for sentence in raw_dataset[:10]:
    print(sentence)

# Concat sentences into single string of chars
# skip blank lines
sentences = [s for s in raw_dataset if s]

# Add the start and stop tokens to each sentence in the file
START_TOKEN = '<s>'
STOP_TOKEN = '</s>'
sentence_list = []
for sentence in sentences:
  sentence_list.append([START_TOKEN] + sentence + [STOP_TOKEN])


for sentence in sentence_list[:5]:
    print(sentence)



['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
['i']
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', 'was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and', 'twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the', 'fire', 'burned', 'brightly', 'and', 'the', 'soft', 'radiance', 'of', 'the', 'incandescent', 'lights', 'in', 'the', 'lilies', 'of', 'silver', 'caught', 'the', 'bubbles', 'that', 'flashed', 'and', 'passed', 'in', 'our', 'glasses', 'our', 'chairs', 'being', 'his', 'patents', 'embraced', 'and', 'caressed', 'us', 'rather', 'than', 'submitted', 'to', 'be', 'sat', 'upon', 'and', 'there', 'was', 'that', 'luxurious', 'after', 'dinner', 'atmosphere', 'when', 'thought', 'roams', 'gracefully', 'free', 'of', 'the', 'trammels', 'of', 'precision', 'and', 'he', 'put', 'it', 'to', 'us', 'in', 'this', 'way', 'marking', 'the', 'points', 'with', 

In [6]:
# Building the dict and the map from the words to integers for the purpose of training.
word2idx={}
idx2word={}
idx=0
for line in sentence_list:
    for word in line:
        if word not in word2idx:
            word2idx[word]=idx
            idx2word[idx]=word
            idx+=1

In [7]:
# there are 4581 unique words.
len(word2idx)

4581

In [8]:
# Transform the words in each sentence to integers
train_data=[]
for line in sentence_list:
    train_data.append([word2idx[word] for word in line])

In [9]:
smm=SimpleMarkovModel(status_num=len(word2idx))
smm.fit(train_data)

In [10]:
# let's see how many entries are 0.
np.sum(smm.P==0)/(smm.P.shape[0]*smm.P.shape[1])
# this is really sparse matrix!!

0.9990304762403064

In [11]:
# Unigrams
# top five most common words in unigrams

from collections import Counter
sentence_list_flatten = list(np.concatenate(sentence_list).flat)
top_5_word_counts = Counter(sentence_list_flatten).most_common(5)
print(top_5_word_counts)

[('the', 2261), ('i', 1267), ('and', 1245), ('of', 1155), ('a', 816)]


In [12]:
# top five common words in stationary distribution
p= smm.predict_prob_distribution(time_steps=50)
idxs = np.argpartition(p, -5)[-5:].tolist()
for idx in idxs:
  print(idx2word[idx])

a
of
and
the
i


In [13]:
# give a default small probability for zero probability entry.
smm.P=np.where(smm.P==0,1.0/smm.P.shape[0],smm.P)

# let's try some random sentences
print(smm.predict_log_joint_prob([word2idx[word] for word in ["time","machine","is","good"]], set_init_prob=p))
print(smm.predict_log_joint_prob([word2idx[word] for word in ["time","good","is","machine"]], set_init_prob=p))


-24.0859057346767
-30.880822608145053


## Generate text data
Markov chain is a generative model so it can be used to generate text. There are two methods: greedy search and beam search.
### Greedy Search
It is the greedy algorithm, so at each step we will go to the state that achieves the maximum probability, i.e.,

$$
S_{\text{next}}^*=\arg\max_{S_{\text{next}}}p(X_{t}=S_{\text{next}}\mid X_{t-1}=S_{\text{current}})
$$   




In [14]:
idx_list=smm.generate_status(search_type="greedy",set_start_status=word2idx['you'] , stop_status=[word2idx[word] for word in ['</s>']])

In [15]:
print([idx2word[idx] for idx in idx_list])

['you', 'know', 'how', 'it', 'was', 'a', 'little', 'people', 'were', 'no', 'doubt']


### Beam Search
With Greedy Search, we took just the single best word at each position and we considered each position in isolation. Once we had identified the best word for that position, we did not examine what came before it (ie. in the previous position), or after it. So it may not find the optimal sequence.

If the goal is to obtain the most likely sequence, we may consider using exhaustive search: exhaustively enumerate all the possible output sequences with their conditional probabilities, and then output the one that scores the highest predicted probability. That will be too expensive!!

Beam search is a compromise between greedy search and exhaustive search.  

**Hyperparameter**:  the beam size $k$.

**Steps**:

-  At time step 1, we select the $k$ tokens with the highest predicted probabilities. Each of them will be the first token of $k$ candidate output sequences, respectively.

- At each subsequent time step, based on the $k$ candidate output sequences at the previous time step, we continue to select $k$ candidate output sequences with the highest predicted probabilities from $k|S|$ possible choices.

- In the end, we obtain the set of final candidate output sequences based on these sequences. we choose the sequence with the highest of the following score as the output sequence:
$$\frac{1}{L^\alpha} \log p(X_1, X_2, \dots, X_L | X_0)= \frac{1}{L^\alpha}\sum_{t=1}^L \log p(X_{t}|X_{t-1}) $$
where $L$ is the length of the final candidate sequence and $\alpha$ is usually set to 0.75. Since a longer sequence has more logarithmic terms in the summation, the term $L^\alpha$ in the denominator penalizes long sequences.


The computational cost of beam search is $O(k|S|L)$. This result is in between that of greedy search and that of exhaustive search. Greedy search can be treated as a special case of beam search arising when the beam size is set to 1.




### Example
The process of beam search (beam size: 2, maximum length of an output sequence: 3).

<img src="https://github.com/yexf308/AppliedStochasticProcess/blob/main/image/beam.png?raw=true" width="800" />




In [16]:
idx_list=smm.generate_status(search_type="beam",set_start_status=word2idx['you'] , stop_status=[word2idx[word] for word in ['</s>']])

In [17]:
print([idx2word[idx] for idx in idx_list])

['you', 'cannot', 'move', 'freely', 'in', 'the', 'time', 'traveller', 's', 'pause', 'that']


# If your dataset is very large
You cannot explicitly construct the matrix, instead we only keep these non-zero entries.



---



---


**Your task:** Complete the code in the ```LanguageModel``` class as follows.

1. (15pt) Implement **Laplace Smoothing** with the smoothing factor =1 in the function ```get_unigram_probability``` and ```get_bigram_probability```. In the bigram model, try to implement it as efficient as possible. **Hint:** For bigrams $(t_1,t_2)$ which do not occur in the sample, what is $p(t_2|t_1)$? For fixed $t_1$, are these probability the same?

2. (15pt) Complete functions ```get_unigram_sentence_log_probability``` and ```get_bigram_sentence_log_probability``` to calculate the log probability of the given sentence. You need to consider the situation that when Laplace Smoothing is true and the situation that when Laplace Smoothing is false.


3. (10pt) To make your language model work better, you will implement linear interpolation smoothing between
unigram, bigram.
\begin{align}
p'(t_2|t_1) = \lambda_1 p(t_2) + \lambda_2 p(t_2|t_1)
\end{align}
where $p'$ represents the smoothed probability, the hyperparameters $\lambda_1, \lambda_2$ are weights on the unigram,
bigram language models, respectively. So $\lambda_1+\lambda_2= 1$.
Complete functions ```get_linear_interpolation_probability``` and ```get_linear_interpolation_sentence_log_probability``` with $\mm\lambda = (\lambda_1, \lambda_2)$ stored in ```linear_interpolation_factors```.

4. (5pt) Testing the smoothed probability with several sentences in the development dataset. You should test when laplace smoothing is True and when laplace smoothing is False. You might find some words in testing dataset are not appeared in training set and you can set a default probability for this situation.  

3. (15pt) Report the **perplexity scores** of the linear interpolation of language model for your training,
and development sets. Report no
more than 5 different sets of $\lambda$. Briefly discuss the experimental results. Putting it all together, report perplexity on the test set, using the hyperparameters that you chose from
the development set. Specify those hyperparameters.

In [18]:
class LanguageModel(object):
    def __init__(self):
        self.unigram_freqs = {}
        self.unigram_corpus_length = 0
        self.num_unique_unigrams = 0

        self.bigram_freqs = {}
        self.bigram_corpus_length = 0
        self.num_unique_bigrams = 0

    def fit_unigram(self, tokens):
        for clist in tokens:
          for cindex in range(len(clist)):
            t = clist[cindex]
            self.unigram_freqs[(t)] = self.unigram_freqs.get((t),0) + 1

        self.unigram_corpus_length = sum(list(self.unigram_freqs.values()))
        self.num_unique_unigrams = len(list(self.unigram_freqs.keys()))

    def fit_bigram(self, tokens):

      for clist in tokens:
        for cindex in range(0, len(clist) - 1):
          t1, t2 =  clist[cindex], clist[cindex+1]
          self.bigram_freqs[(t1, t2)] = self.bigram_freqs.get((t1, t2), 0) + 1

      self.bigram_corpus_length = sum(list(self.bigram_freqs.values()))
      self.num_unique_bigrams = len(list(self.bigram_freqs.keys()))

    def get_unigram_probability(self, word):
      prob_numerator = self.unigram_freqs.get(word, 0)
      prob_denominator = self.unigram_corpus_length

      prob = float(prob_numerator) / float(prob_denominator)
      return prob

    def get_bigram_probability(self, bigram):
      t1, t2 = bigram
      prob_numerator   = self.bigram_freqs.get((t1, t2), 0)
      prob_denominator = self.unigram_freqs.get(t1, 0)
      if prob_denominator == 0:
          print(f"Error: 'get_bigram_probability()' has a denominator of 0 for {bigram}")
          return float('inf')

      prob = float(prob_numerator) / float(prob_denominator)
      return prob




In [ ]:
len(list(LM.unigram_freqs.keys()))

4581

In [ ]:
LM.fit_bigram(train_data)
print(LM.bigram_freqs)

{(0, 1): 76, (1, 2): 204, (2, 3): 78, (3, 4): 2, (4, 5): 2, (5, 6): 2, (6, 7): 2, (7, 8): 2, (0, 9): 108, (9, 8): 4, (2, 10): 120, (10, 11): 2, (11, 12): 2, (12, 13): 8, (13, 14): 12, (14, 15): 8, (15, 16): 2, (16, 17): 2, (17, 18): 8, (18, 19): 4, (19, 20): 4, (20, 21): 4, (21, 22): 4, (22, 23): 2, (23, 24): 2, (24, 25): 2, (25, 17): 2, (17, 26): 8, (26, 27): 2, (27, 28): 2, (28, 29): 4, (29, 30): 4, (30, 31): 2, (31, 32): 2, (32, 31): 2, (31, 27): 8, (27, 33): 2, (33, 34): 2, (34, 35): 2, (35, 21): 6, (21, 36): 2, (36, 31): 2, (31, 37): 2, (37, 1): 2, (1, 38): 24, (38, 39): 2, (39, 40): 2, (40, 31): 6, (31, 1): 218, (1, 41): 4, (41, 42): 2, (42, 19): 2, (19, 1): 618, (1, 43): 2, (43, 44): 2, (44, 45): 2, (45, 1): 338, (1, 46): 2, (46, 19): 2, (19, 47): 4, (47, 48): 2, (48, 1): 6, (1, 49): 2, (49, 50): 2, (50, 51): 2, (51, 31): 2, (31, 52): 6, (52, 45): 2, (45, 53): 10, (53, 54): 2, (54, 53): 2, (53, 55): 2, (55, 56): 2, (56, 27): 2, (27, 57): 2, (57, 58): 2, (58, 31): 2, (31, 59): 4,

In [ ]:
LM.get_bigram_probability((4,1))

0.6213592233009708